In [0]:
## desmontar acceso directo: unmount
dbutils.fs.unmount("/mnt/proyectofinal")

/mnt/proyectofinal has been unmounted.


True

In [0]:
%scala

val storageAccount = "corteztenorio" //nombre de la cuenta de almacenamiento
val container = "proyectofinal" // nombre del contenedor
val sasKey = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-10-11T09:59:01Z&st=2024-09-27T01:59:01Z&spr=https&sig=SmZYHYeHyKXKYy9fNrQJgvdm4gXRciY1cUtX7qqI6yI%3D" // llave de acceso

val conf = "fs.azure.sas." + container + "." + storageAccount + ".blob.core.windows.net"

//montar acceso directo: mount
dbutils.fs.mount(source="wasbs://" + container + "@" + storageAccount + ".blob.core.windows.net", mountPoint="/mnt/" + container, extraConfigs=Map(conf -> sasKey))

storageAccount: String = corteztenorio
container: String = proyectofinal
sasKey: String = sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-10-11T09:59:01Z&st=2024-09-27T01:59:01Z&spr=https&sig=SmZYHYeHyKXKYy9fNrQJgvdm4gXRciY1cUtX7qqI6yI%3D
conf: String = fs.azure.sas.proyectofinal.corteztenorio.blob.core.windows.net
res13: Boolean = true

In [0]:
%fs ls /mnt/proyectofinal/capa_plata/

path,name,size,modificationTime
dbfs:/mnt/proyectofinal/capa_plata/Categoria.parquet/,Categoria.parquet/,0,0
dbfs:/mnt/proyectofinal/capa_plata/Producto.parquet/,Producto.parquet/,0,0
dbfs:/mnt/proyectofinal/capa_plata/SubCategoria.parquet/,SubCategoria.parquet/,0,0


In [0]:
## Eliminar si existe Producto.parque
spark.sql("DROP TABLE IF EXISTS s_producto")

## Ingesta archivo Producto.parquet
producto = (spark.read.parquet("/mnt/proyectofinal/capa_plata/Producto.parquet/"))
producto.createTempView("s_producto")

In [0]:
## Eliminar si existe Categoria.parquet
spark.sql("DROP TABLE IF EXISTS s_categoria")

## Ingesta archivo Categoria.parquet
producto = (spark.read.parquet("/mnt/proyectofinal/capa_plata/Categoria.parquet/"))
producto.createTempView("s_categoria")

In [0]:
## Eliminar si existe Subcategoria.parquet
spark.sql("DROP TABLE IF EXISTS s_subcategoria")

## Ingesta archivo SubCategoria.parquet
producto = (spark.read.parquet("/mnt/proyectofinal/capa_plata/SubCategoria.parquet/"))
producto.createTempView("s_subcategoria")

In [0]:
%sql

-- Si tabla Fact_Producto ya existe, la elimino
DROP TABLE IF EXISTS Fact_Product;

CREATE TEMP VIEW Fact_Product AS 
SELECT 
    P.Cod_Producto, 
    P.Nombre AS Producto, 
    S.Nombre AS SubCategoria, 
    C.Nombre AS Categoria
FROM 
    s_producto P
JOIN 
    s_categoria S ON P.Cod_SubCategoria = S.Cod_Categoria
JOIN 
    s_categoria C ON S.Cod_Categoria = C.Cod_Categoria

In [0]:
dfFactProducto = spark.table("Fact_Product")
dfFactProducto.write.mode("overwrite").parquet("/mnt/proyectofinal/capa_oro/FactProducto.parquet/")